In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# You can only unzip a zipped tuple once before it is removed from the computer's memory. 
# Make sure you unzip the latitudes and longitudes into the coordinates list before moving on

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# use citipy to find the nearest city
# The citipy module finds the nearest city to the latitude and longitude pair with a population of 500 or more.

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

622

In [7]:
# Import the requests library.
import requests

# Import the API key.
import sys
sys.path.append('/Users/katiemtroll/Desktop/API_Keys')
from api_keys import key_open_weather
api_key = key_open_weather

In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

In [9]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 43.63,
  'feels_like': 43.63,
  'temp_min': 38.16,
  'temp_max': 47.01,
  'pressure': 1014,
  'humidity': 89},
 'visibility': 10000,
 'wind': {'speed': 1.01, 'deg': 291, 'gust': 3},
 'clouds': {'all': 90},
 'dt': 1640990992,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1640952800,
  'sunset': 1640985665},
 'timezone': -18000,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [10]:
# Get the JSON data.
boston_data = city_weather.json()

In [11]:
boston_data["sys"]

{'type': 2,
 'id': 2013408,
 'country': 'US',
 'sunrise': 1640952800,
 'sunset': 1640985665}

In [12]:
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

42.3584 -71.0598 47.01 89 90 1.01


In [13]:
# convert the datetime stamp
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date)

datetime.datetime(2021, 12, 31, 22, 49, 52)

In [14]:
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-12-31 22:49:52'

In [15]:
# now let's get the data for all of our cities
# We will need to do the following:
# 1. Import our dependencies and initialize counters and an empty list that will hold the weather data.
# 2. Loop through the cities list.
# 3. Group the cities in sets of 50 to log the process as we find the weather data for each city.
# - Two counters will be needed here: one to log the city count from 1 to 50, and another for the sets.
# 4. Build the city_url or endpoint for each city.
# 5. Log the URL and the record and set numbers.
# 6. Make an API request for each city.
# 7. Parse the JSON weather data for the following:
# - City, country, and date
# - Latitude and longitude
# - Maximum temperature
# - Humidity
# - Cloudiness
# - Wind speed
# 8. Add the data to a list in a dictionary format and then convert the list to a DataFrame.

In [16]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [17]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
# enumerate to get the index in a way that can log in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    # time sleep to prevent time-out error (only 60 calls per minute allowed on OpenWeather free tier)
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | cape town
Processing Record 3 of Set 1 | whyalla
Processing Record 4 of Set 1 | port-cartier
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | mahebourg
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | kapaa
Processing Record 9 of Set 1 | santa eulalia del rio
City not found. Skipping...
Processing Record 10 of Set 1 | chuy
Processing Record 11 of Set 1 | mapastepec
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | marzuq
Processing Record 14 of Set 1 | khatanga
Processing Record 15 of Set 1 | maroua
Processing Record 16 of Set 1 | mastic beach
Processing Record 17 of Set 1 | hualmay
Processing Record 18 of Set 1 | mar del plata
Processing Record 19 of Set 1 | merauke
Processing Record 20 of Set 1 | ondarroa
Processing Record 21 of Set 1 | bredasdorp
Processing Record 22 of Set 1 | nuuk
Process

Processing Record 38 of Set 4 | krasnorechenskiy
Processing Record 39 of Set 4 | mount gambier
Processing Record 40 of Set 4 | bambous virieux
Processing Record 41 of Set 4 | pokhara
Processing Record 42 of Set 4 | nouadhibou
Processing Record 43 of Set 4 | cherskiy
Processing Record 44 of Set 4 | moyale
Processing Record 45 of Set 4 | hermanus
Processing Record 46 of Set 4 | vanimo
Processing Record 47 of Set 4 | alofi
Processing Record 48 of Set 4 | chapais
Processing Record 49 of Set 4 | marawi
Processing Record 50 of Set 4 | kaniama
Processing Record 1 of Set 5 | esperance
Processing Record 2 of Set 5 | duki
Processing Record 3 of Set 5 | maniitsoq
Processing Record 4 of Set 5 | kailua
Processing Record 5 of Set 5 | port blair
Processing Record 6 of Set 5 | barranca
Processing Record 7 of Set 5 | gijon
Processing Record 8 of Set 5 | north bend
Processing Record 9 of Set 5 | bubaque
Processing Record 10 of Set 5 | camalu
Processing Record 11 of Set 5 | tsihombe
City not found. Skipp

Processing Record 27 of Set 8 | santa cruz
Processing Record 28 of Set 8 | kagoro
Processing Record 29 of Set 8 | umm kaddadah
Processing Record 30 of Set 8 | vetovo
Processing Record 31 of Set 8 | villarrubia de los ojos
Processing Record 32 of Set 8 | skagastrond
City not found. Skipping...
Processing Record 33 of Set 8 | susa
Processing Record 34 of Set 8 | lemesos
City not found. Skipping...
Processing Record 35 of Set 8 | basoko
Processing Record 36 of Set 8 | bathsheba
Processing Record 37 of Set 8 | rindal
Processing Record 38 of Set 8 | kapuskasing
Processing Record 39 of Set 8 | florence
Processing Record 40 of Set 8 | porosozero
Processing Record 41 of Set 8 | novo aripuana
Processing Record 42 of Set 8 | gayeri
Processing Record 43 of Set 8 | havelock
Processing Record 44 of Set 8 | cairns
Processing Record 45 of Set 8 | kot samaba
Processing Record 46 of Set 8 | praya
Processing Record 47 of Set 8 | dikson
Processing Record 48 of Set 8 | trairi
Processing Record 49 of Set 8

Processing Record 12 of Set 12 | otane
Processing Record 13 of Set 12 | gazojak
Processing Record 14 of Set 12 | najran
Processing Record 15 of Set 12 | kargasok
Processing Record 16 of Set 12 | ursulo galvan
Processing Record 17 of Set 12 | ormara
Processing Record 18 of Set 12 | goderich
Processing Record 19 of Set 12 | sagarejo
Processing Record 20 of Set 12 | auki
Processing Record 21 of Set 12 | kompaniyivka
Processing Record 22 of Set 12 | tabas
Processing Record 23 of Set 12 | marcona
City not found. Skipping...
Processing Record 24 of Set 12 | balabac
Processing Record 25 of Set 12 | porto novo
Processing Record 26 of Set 12 | half moon bay
Processing Record 27 of Set 12 | mosquera
Processing Record 28 of Set 12 | damoh
Processing Record 29 of Set 12 | axim
Processing Record 30 of Set 12 | sembe
Processing Record 31 of Set 12 | srednekolymsk
Processing Record 32 of Set 12 | sabha
Processing Record 33 of Set 12 | kadykchan
City not found. Skipping...
Processing Record 34 of Set 

In [19]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,77.76,73,19,11.65,PF,2021-12-31 22:50:58
1,Cape Town,-33.9258,18.4232,63.88,76,20,1.99,ZA,2021-12-31 22:51:04
2,Whyalla,-33.0333,137.5833,78.69,41,100,3.44,AU,2021-12-31 22:53:05
3,Port-Cartier,50.0334,-66.8654,28.51,89,76,2.68,CA,2021-12-31 22:53:06
4,Qaanaaq,77.4840,-69.3632,-0.92,67,95,6.80,GL,2021-12-31 22:51:01
5,Mahebourg,-20.4081,57.7000,75.49,83,75,3.44,MU,2021-12-31 22:53:06
6,Ushuaia,-54.8000,-68.3000,47.86,53,40,25.32,AR,2021-12-31 22:48:45
7,Kapaa,22.0752,-159.3190,77.47,73,87,3.00,US,2021-12-31 22:53:07
8,Chuy,-33.6971,-53.4616,72.39,82,29,14.50,UY,2021-12-31 22:50:56
9,Mapastepec,15.4333,-92.9000,82.20,67,31,6.17,MX,2021-12-31 22:53:08


In [23]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Rikitea,PF,2021-12-31 22:50:58,-23.1203,-134.9692,77.76,73,19,11.65
1,Cape Town,ZA,2021-12-31 22:51:04,-33.9258,18.4232,63.88,76,20,1.99
2,Whyalla,AU,2021-12-31 22:53:05,-33.0333,137.5833,78.69,41,100,3.44
3,Port-Cartier,CA,2021-12-31 22:53:06,50.0334,-66.8654,28.51,89,76,2.68
4,Qaanaaq,GL,2021-12-31 22:51:01,77.4840,-69.3632,-0.92,67,95,6.80
5,Mahebourg,MU,2021-12-31 22:53:06,-20.4081,57.7000,75.49,83,75,3.44
6,Ushuaia,AR,2021-12-31 22:48:45,-54.8000,-68.3000,47.86,53,40,25.32
7,Kapaa,US,2021-12-31 22:53:07,22.0752,-159.3190,77.47,73,87,3.00
8,Chuy,UY,2021-12-31 22:50:56,-33.6971,-53.4616,72.39,82,29,14.50
9,Mapastepec,MX,2021-12-31 22:53:08,15.4333,-92.9000,82.20,67,31,6.17


In [24]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")